<a href="https://colab.research.google.com/github/tasyaa04/datascience/blob/main/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
###DO NOT EDIT THIS CODE
################################################################################################################################

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from PIL import Image
import matplotlib.pyplot as plt

# GPUs are 3x faster than CPU. Better to use if it is available 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define Loss Function
loss_function = nn.CrossEntropyLoss()

# This function returns the number of parameters in the model
def num_params(model):
  return sum([p.numel() for p in model.parameters()])

# Define a Training Function. This function will: compute the forward pass, backpropagate,
# update the weights, and repeat the steps for a given number of epochs. At each epoch, 
# it will output the training loss and test loss at every step
def train(epochs, model, trainloader, testloader, optimizer, loss_function):
  for epoch in range(epochs):
    loss_epoch = np.array([])
    train_correct, train_total = 0, 0
    test_correct, test_total = 0, 0

    for data, labels in trainloader:
      # convert into GPU objects if needed
      input_data = data.to(device)
      labels = labels.to(device)

      # forward pass
      predict = model(input_data)
      
      # backward pass
      loss = loss_function(predict, labels)
      optimizer.zero_grad()
      loss.backward()

      # update parameters (weights and biases)
      optimizer.step()

      # store progress
      loss_epoch = np.append(loss_epoch, loss.item())

    # evaluate test accuracy
    for data, labels in testloader:
      input_data = data.to(device)
      labels = labels.to(device)
      predict = model(input_data)
      for i, out in enumerate(predict):
        pred = torch.argmax(out)
        if pred == labels[i]:
          test_correct+=1
        test_total+=1

    test_accuracy = test_correct/test_total    
  
    print('epoch [{}/{}], training loss:{:.4f}, test accuracy:{:.4f}'.format(epoch+1, epochs, np.mean(loss_epoch), test_accuracy))
################################################################################################################################

# **Load Dataset**

Available datasets are: MNIST, CIFAR10

## MNIST

In [18]:
# download and load data
batch_size = 512

# download and transform train dataset
train_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist_data', download=True, train=True, transform=transforms.Compose([
                                                transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                ])), batch_size=batch_size, shuffle=True)

# download and transform test dataset
test_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist_data', download=True, train=False, transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])), batch_size=batch_size, shuffle=True)

In [19]:
# it is a good idea to take a look at the data. Here we see it is a 28x28 grayscale image
for data, labels in train_loader:
  print(data[0].size())
  break

torch.Size([1, 28, 28])


# **Build a Network and Define Hyperparameters**

In [37]:
########################
####                #### 
#### EDIT THIS CELL ####
####                ####
########################


learning_rate = 17e-3
weight_decay = 10e-5
n_epochs = 10

# neural network
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    ### Define Layers
    self.conv1 = nn.Conv2d(1, 3, 3, padding=1)
    self.fc1 = nn.Linear(2352, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = F.sigmoid(x)
    x = F.relu(x)
    return x
  
# Every time you edit the neural network, you'll have to update this cell
# Create model object
model = NeuralNetwork().to(device)

# Loads Adam optimizer, which implements a version of gradient descent
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [38]:
# check the structure of your network
print(model)

# apply your model to a single input. This helps check that 
# the dimensions are correct
model(torch.rand(1,1,28,28, device=device))

NeuralNetwork(
  (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2352, out_features=10, bias=True)
)


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[0.5002, 0.5295, 0.5027, 0.5013, 0.5019, 0.5000, 0.5487, 0.5000, 0.5000,
         0.5444]], device='cuda:0', grad_fn=<ReluBackward0>)

# **Train and Validate**

In [41]:
train(n_epochs, model, train_loader, test_loader, optimizer, loss_function)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch [1/10], training loss:1.8895, test accuracy:0.9546
epoch [2/10], training loss:1.8897, test accuracy:0.9530
epoch [3/10], training loss:1.8892, test accuracy:0.9564
epoch [4/10], training loss:1.8886, test accuracy:0.9524
epoch [5/10], training loss:1.8884, test accuracy:0.9557
epoch [6/10], training loss:1.8889, test accuracy:0.9513
epoch [7/10], training loss:1.8884, test accuracy:0.9562
epoch [8/10], training loss:1.8886, test accuracy:0.9566
epoch [9/10], training loss:1.8886, test accuracy:0.9529
epoch [10/10], training loss:1.8903, test accuracy:0.9578
